# Imports

In [88]:
import random
import copy
import math
import matplotlib.pyplot as plt


# Graph dicts

In [89]:
# A dictionary mapping graph filenames to their known chromatic numbers
GRAPH_DICT = {
    #'myciel3.col': 4,
    'queen8_8.col': 9
    #'le450_15b.col': 15,
}
PRINT_EACH = 1  # Number of generations between print statements
CHROMATIC_MARGIN = 5 # Margin above known chromatic number to start with

# Declaring Functions

## Parse graph from .col file (Dataset)

In [90]:
def parse_col_file(filepath):
    adj = {}
    num_vertices = 0

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('p'):
                _, _, n, _ = line.split()
                num_vertices = int(n)
                adj = {i: set() for i in range(1, num_vertices + 1)}

            elif line.startswith('e'):
                _, u, v = line.split()
                u, v = int(u), int(v)
                adj[u].add(v)
                adj[v].add(u)
    return adj, num_vertices


## Fitness and Conflict Evaluation

In [91]:
def count_conflicts(adj, coloring):
    conflicts = 0
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts += 1
    return conflicts

In [92]:
def num_colors_used(coloring):
    return len(set(coloring.values()))

In [93]:
def fitness(adj, coloring, alpha=1, beta=1):
    return alpha * count_conflicts(adj, coloring) + beta * num_colors_used(coloring)

## Population Init

In [94]:
def random_coloring(num_vertices, max_colors):
    return {i: random.randint(1, max_colors) for i in range(1, num_vertices + 1)}

In [95]:
def initialize_population(pop_size, num_vertices, max_colors):
    return [random_coloring(num_vertices, max_colors) for _ in range(pop_size)]

## Tournament Selection

In [96]:
def tournament_selection(population, adj, k=3):
    candidates = random.sample(population, k)
    return min(candidates, key=lambda c: fitness(adj, c))

## Crossover (Uniform)

In [97]:
def uniform_crossover(parent1, parent2):
    child = {}
    for v in parent1:
        child[v] = parent1[v] if random.random() < 0.5 else parent2[v]
    return child


## Mutation and conflict fix

In [98]:
def mutate(coloring, adj, max_colors, mutation_rate=0.02):
    for v in coloring:
        if random.random() < mutation_rate:
            coloring[v] = random.randint(1, max_colors)

    # Extra: fix conflicts
    for u in adj:
        for v in adj[u]:
            if coloring[u] == coloring[v]:
                coloring[v] = random.randint(1, max_colors)

# Genetic Algorithm

## Definition of the Genetic Function

In [99]:
def genetic_algorithm(
    adj,
    num_vertices,
    max_colors,
    pop_size=100,
    generations=1000,
    elitism_rate=0.05,
    mutation_rate=0.02
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number 

    population = initialize_population(pop_size, num_vertices, max_colors) # Initial population
    elite_size = int(pop_size * elitism_rate) # Number of elites
    # initialize best solution tracking
    best_solution = None
    best_fitness = float('inf')
    found_feasible = False
    conflicts = []
    
    # main GA loop
    for gen in range(generations):
        population.sort(key=lambda c: fitness(adj, c))  # Sort by fitness
        elites = population[:elite_size]               # Select elites

        current_best = population[0]
        current_fitness = fitness(adj, current_best)
        current_conflicts = count_conflicts(adj, current_best)
        current_colors = num_colors_used(current_best)

        # Update global best
        if current_fitness < best_fitness:                  # New best found
            best_solution = copy.deepcopy(current_best)     # Update best solution
            best_fitness = current_fitness                  # Update best fitness

        if gen % PRINT_EACH == 0:   # Print every set number of generations
            conflicts.append(current_conflicts)
            print(
                f"Gen {gen} | Best fitness: {best_fitness} | "
                f"Conflicts: {count_conflicts(adj, best_solution)} | "
                f"Colors: {num_colors_used(best_solution)}"
            )

        # First feasible solution (any number of colors)
        if current_conflicts == 0 and not found_feasible:
            print(f"Feasible solution found at generation {gen}")
            found_feasible = True

        # Early stopping condition
        if current_conflicts == 0 and current_colors <= chromatic_number:   # Optimal solution found, colors equal chromatic number and no conflicts
            print(
                f"Optimal solution found at generation {gen} | "
                f"Colors: {current_colors}"
            )
            return current_best, conflicts

        new_population = elites.copy()  # Start new population with elites (unchanged)

        while len(new_population) < pop_size:   # Fill the rest of the population using tournament selection, crossover, and mutation
            p1 = tournament_selection(population, adj)
            p2 = tournament_selection(population, adj)

            child = uniform_crossover(p1, p2)
            mutate(child, adj, max_colors, mutation_rate)

            new_population.append(child)

        population = new_population

    return best_solution, conflicts


## Usage of the Genetic Algorithm

In [100]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 55)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 55)

        solution, conflicts = genetic_algorithm(
            adj=adj,
            num_vertices=n,
            max_colors=max_colors,
            pop_size=200,
        )

        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))



queen8_8.col: n=64, chromatic_number=9 max_colors=14
Gen 0 | Best fitness: 50 | Conflicts: 36 | Colors: 14
Gen 1 | Best fitness: 24 | Conflicts: 10 | Colors: 14
Gen 2 | Best fitness: 23 | Conflicts: 9 | Colors: 14
Gen 3 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 4 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 5 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 6 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 7 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 8 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 9 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 10 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 11 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 12 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 13 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 14 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 15 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 16 | Best fitness: 22 | Conflicts: 8 | Colors: 14
Gen 17 | Best fitness: 22 | Conflicts

# Tabu Search

## Implementation of an optimized conflict storage
Using a data structure reduces the amount of computations since the tabu search requires not only the amount of conflicts but where have they appeared in order to generate meaningful neighborhood moves, because only recoloring conflicted vertices can reduce conflicts.
The total conflict count alone provides no structural information about where the problem is, making targeted local search impossible.

In [101]:
def conflicting_vertices(adj, coloring):
    conflicts = set()
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts.add(u)
                conflicts.add(v)
    return list(conflicts)


## Implementation of the algorithm

In [102]:
def tabu_search(
    adj,
    num_vertices,
    max_colors,
    max_iterations=20000,
    tabu_tenure=7,
    aspiration=True,
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)

    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors

    tabu_list = {}
    iteration = 0

    while iteration < max_iterations:

        conflicted = conflicting_vertices(adj, current)
        best_move = None
        best_score = (float("inf"), float("inf"))

        for v in conflicted:
            original_color = current[v]

            for c in range(1, max_colors + 1):
                if c != original_color:

                    current[v] = c
                    new_conflicts = count_conflicts(adj, current)
                    new_colors = num_colors_used(current)

                    score = (new_conflicts, new_colors)
                    delta_better = score < best_score

                    move = (v, c)
                    tabu = move in tabu_list and tabu_list[move] > iteration

                    if tabu and aspiration:
                        if (new_conflicts < best_conflicts or
                           (new_conflicts == best_conflicts and new_colors < best_colors)):
                            tabu = False

                    if not tabu and delta_better:
                        best_score = score
                        best_move = (v, c, new_conflicts, new_colors)

                    current[v] = original_color

        if best_move is not None:
            v, c, new_conflicts, new_colors = best_move
            old_color = current[v]

            current[v] = c
            current_conflicts = new_conflicts
            current_colors = new_colors

            tabu_list[(v, old_color)] = iteration + tabu_tenure

            # Update global best using lexicographic objective
            if (current_conflicts < best_conflicts or
               (current_conflicts == best_conflicts and current_colors < best_colors)):
                best = copy.deepcopy(current)
                best_conflicts = current_conflicts
                best_colors = current_colors

        if iteration % PRINT_EACH == 0:
            print(
                f"Iter {iteration} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )

        iteration += 1

    return best


## Usage of Tabu Search

In [103]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 50)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 50)

        solution = tabu_search(
        adj=adj,
        num_vertices=n,
        max_colors=max_colors,
        max_iterations=2000,
        tabu_tenure=7,
    )

        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))    



queen8_8.col: n=64, chromatic_number=9 max_colors=14
Iter 0 | Conflicts: 56 | Colors: 14 | Best colors: 14
Iter 1 | Conflicts: 51 | Colors: 14 | Best colors: 14
Iter 2 | Conflicts: 47 | Colors: 14 | Best colors: 14
Iter 3 | Conflicts: 43 | Colors: 14 | Best colors: 14
Iter 4 | Conflicts: 40 | Colors: 14 | Best colors: 14
Iter 5 | Conflicts: 37 | Colors: 14 | Best colors: 14
Iter 6 | Conflicts: 34 | Colors: 14 | Best colors: 14
Iter 7 | Conflicts: 31 | Colors: 14 | Best colors: 14
Iter 8 | Conflicts: 28 | Colors: 14 | Best colors: 14
Iter 9 | Conflicts: 25 | Colors: 14 | Best colors: 14
Iter 10 | Conflicts: 23 | Colors: 14 | Best colors: 14
Iter 11 | Conflicts: 21 | Colors: 14 | Best colors: 14
Iter 12 | Conflicts: 19 | Colors: 14 | Best colors: 14
Iter 13 | Conflicts: 17 | Colors: 14 | Best colors: 14
Iter 14 | Conflicts: 15 | Colors: 14 | Best colors: 14
Iter 15 | Conflicts: 13 | Colors: 14 | Best colors: 14
Iter 16 | Conflicts: 11 | Colors: 14 | Best colors: 14
Iter 17 | Conflicts: 9

# Simmulated Annealing

## Algorithm implementation

In [104]:
VERTICES = list(adj.keys())

def simulated_annealing(
    adj,
    num_vertices,
    max_colors,
    max_iterations=50000,
    initial_temp=10.0,
    cooling_rate=0.9995,
    conflict_weight=1000,
):
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)

    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors

    temperature = initial_temp
    iteration = 0

    while iteration < max_iterations and temperature > 1e-6:

        # Pick a vertex biased toward conflicts
        conflicted = conflicting_vertices(adj, current)
        if len(conflicted) > 0:
            v = random.choice(conflicted)
        else:
            v = random.choice(VERTICES)

        original_color = current[v]
        new_color = random.randint(1, max_colors)

        if new_color != original_color:

            current[v] = new_color
            new_conflicts = count_conflicts(adj, current)
            new_colors = num_colors_used(current)

            delta = (
                (new_conflicts - current_conflicts) * conflict_weight
                + (new_colors - current_colors)
            )

            accept = False
            if delta <= 0:
                accept = True
            else:
                prob = math.exp(-delta / temperature)
                accept = random.random() < prob

            if accept:
                current_conflicts = new_conflicts
                current_colors = new_colors

                if (
                    current_conflicts < best_conflicts or
                    (current_conflicts == best_conflicts and current_colors < best_colors)
                ):
                    best = copy.deepcopy(current)
                    best_conflicts = current_conflicts
                    best_colors = current_colors
            else:
                current[v] = original_color

        temperature *= cooling_rate
        iteration += 1

        if iteration % 1000 == 0:
            print(
                f"Iter {iteration} | "
                f"T={temperature:.4f} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )

    return best


## Running the algorithm


In [105]:
if __name__ == "__main__":
    adj, n = parse_col_file("queen5_5.col")

    MAX_COLORS = 7

    solution = simulated_annealing(
        adj=adj,
        num_vertices=n,
        max_colors=MAX_COLORS,
    )

    print("Final solution:")
    print("Conflicts:", count_conflicts(adj, solution))
    print("Colors used:", num_colors_used(solution))


FileNotFoundError: [Errno 2] No such file or directory: 'queen5_5.col'